<a href="https://colab.research.google.com/github/Seph-iroth/RoboLearning/blob/main/mecs6616_Spring2024_Project4_JL6080.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MECS6616 Spring 2024 - Project 4**

# **Introduction**

***IMPORTANT:***
- **Before starting, make sure to read the [Assignment Instructions](https://courseworks2.columbia.edu/courses/197115/pages/assignment-instructions) page on Courseworks to understand the workflow and submission requirements for this project.**

**FOR PROJECT 4!!!**
- Apart from the link to your notebook, you are also required to submit the collected data file `data.pkl` and your chosen model checkpoint `dynamic.pth` to Coursework. You should put the link to your notebook in the "Comment" section of your submission.
- You can find a walkthrough of this project on Canvas under "Video Library" (03/26 Lecture), where we discuss key implementation details.

## Project Setup

In [ ]:
# DO NOT CHANGE

# After running this cell, the folder 'mecs6616_sp23_project4' will show up in the file explorer on the left (click on the folder icon if it's not open)
# It may take a few seconds to appear
!git clone https://github.com/roamlab/mecs6616_sp24_project4.git

fatal: destination path 'mecs6616_sp24_project4' already exists and is not an empty directory.


In [ ]:
# DO NOT CHANGE

# move all needed files into the working directory. This is simply to make accessing files easier
!mv /content/mecs6616_sp24_project4/* /content/

In [ ]:
# Using ray for data collection is optional
!pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 MB 3.6 MB/s eta 0:00:00


# Starter Code Explanation

This project uses the same environment as in project 3 which consists of an n linked robot arm. We provide code for teacher dynamics and the controller. We also provide scripts for collecting data, skeleton classes for implementing student dynamics and finally training and evaluating the model.

Similar to last time we define a Robot class inside `robot.py` which provides the interface for controlling the robot arm i.e it provides you with some functions to set/get the state and set the action for the arm and take a step using the `Robot.advance()` method. The state of the arm is a 2n-dimensional vector : n joint positions [rad] + n joint velocities [rad/s] and the action is defined as the n torques (in N-m) applied to n joints respectively.

In `arm_dynamic_base.py` we define the base class for forward dynamics for the arm. The ground truth forward dynamics are defined in `arm_dynamics_teacher.py`. The student dynamics which internally uses a neural network model is defined in `arm_dynamics_student.py`

You are welcome to look in-depth to understand how the ground truth forward dynamics is computed for an arm, given its number of links, link mass, and viscous friction of the environment - however this is not necessary to successfully complete this assignment.


The `models.py` file provides the base class for neural network to learn forward model and also contains skeleton code for you to implement the network architecture for arms with different number of links. We will revisit this again in when providing instructions for training below.

## Part 1: Implement Model Predictive Control

You will implement this controller by completing the MPC class. Specifically, you will implement the compute_action() method by following the algorithm discussed in the lecture. As with previous projects you are free to implement additional methods as needed or change the initialization if need be. While scoring your controller, you will be creating an instance of the MPC class and passing it to the scoring function so ensure that the arguments to the compute_action method remain the same.

Although you do not need to understand how the ArmDynamicsTeacher class works, you could use the compute_fk() method from the class. This will allow you to convert from the state value (represented as array of shape (2*n, 1) where n is num_links) to final end effector position (x, y position of the end effector). Similary we can also compute the velocity of the end effector with the code below:
```
pos_ee = dynamics.compute_fk(state)
vel_ee = dynamics.compute_vel_ee(state)
```


In [ ]:
# @title Currently working on
from collections.abc import Sequence
from collections import defaultdict
import numpy as np
from scipy.optimize import minimize
# import torch as pytorch
from autograd import grad
import random
from copy import deepcopy
from collections import defaultdict
import robot as Robot

import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher

from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
import random


# def TwoLink(dynamics, state, goal, action):
# def ThreeLink(dynamics, state, goal, action):


class MPC:

    def __init__(self,alpha = 150,beta = 4, planning_horizon=65,OPTM_iter=200,bound=2,control_horizon = 20):
        self.control_horizon = control_horizon  # Time steps for control decisions
        self.planning_horizon = planning_horizon  # Time steps for planning future actions
        self.position_weight = alpha # Weight for positional error
        self.velocity_weight =  beta # Weight for velocity penalty
        self.gamma = 0  # Weight for control effort
        self.delta = 0  # Weight for action smoothness
        self.max_velocity = 0  # Maximum allowable velocity
        self.OPTM_iter = OPTM_iter
        self.bound = bound
        random.seed(88488)
        np.random.seed(88481)

    def compute_action(self, dynamics, state, goal, action):
        current_dist_to_goal = np.linalg.norm(dynamics.compute_fk(state)-goal)
        num_action = dynamics.get_action_dim()
        self.num_link = dynamics.get_action_dim()

        current_state = state
        if action.shape[0]==1 and action.shape[1]==1:
          current_state = np.zeros((3,), dtype=np.float32)

        current_action = deepcopy(action)

        best_ASQ = np.array([])
        best_Cost = float('inf')


        given_action_Sequence = self.newASQ(current_action, self.planning_horizon)
        cost = self.compute_cost(dynamics, state, goal, given_action_Sequence)
        # if num_action == 3:
        if num_action == 3:
           planning_horizon=120,
           OPTM_iter=200
        for n in range(self.OPTM_iter):
          if num_action == 1:
            now_ASQ = self.oneLink(dynamics, state, goal, given_action_Sequence)
          if num_action == 2:
            now_ASQ = self.TwoLink(dynamics, state, goal, given_action_Sequence)
          if num_action == 3:
            now_ASQ = self.ThreeLink(dynamics, state, goal, given_action_Sequence)
          cost = self.roll_out(dynamics,state,now_ASQ,goal)

          if cost <= best_Cost:
              best_ASQ = now_ASQ
              best_Cost = cost

        return best_ASQ[0]

    def newASQ(self,action, planning_horizon):
    # Repeat the 'action' array 'planning_horizon' times along the first axis
      actions_sequence = np.tile(action, (planning_horizon, 1))
      return actions_sequence

    def roll_out(self, dynamics, initial_state, action_sequence, goal):
        state = initial_state
        for action in action_sequence:
            if action.shape[0] !=3:
              action = np.zeros((self.num_link,), dtype=np.float32)
            state = dynamics.advance(state ,action)# state(4, 1) action(1,)
        cost = self.compute_cost(dynamics, state, goal, action_sequence)
        return cost

    def compute_cost(self, dynamics, state, goal, action_sequence):
        e_pos = np.linalg.norm(dynamics.compute_fk(state) - goal)
        vel_ee = dynamics.compute_vel_ee(state)
        vel_magnitude = np.linalg.norm(vel_ee)
        e_vel = max(0, vel_magnitude - self.max_velocity)
        e_u = np.sum(action_sequence**2)
        e_smooth = np.sum(np.diff(action_sequence, axis=0)**2)
        total_cost = (self.position_weight * e_pos ** 2)
        + (self.velocity_weight * vel_ee)
        + (self.gamma * e_u)
        + (self.delta * e_smooth)
        return total_cost
    def oneLink(self, dynamics, state, goal, given_action_Sequence):

      self.control_horizon = 30
      self.planning_horizon=45
      self.OPTM_iter = 40

      bound1 ,bound2, bound3= 0.10, 0.17 , 0.17
      # bound1 ,bound2, bound3= self.bound, self.bound , self.bound
      if np.linalg.norm(dynamics.compute_fk(state)) < 0.1:
        self.control_horizon = 10
        # self.planning_horizon = 80
      delta_1,delta_2,delta_3 =  random.uniform(-bound1, bound1),random.uniform(-bound2, bound2),random.uniform(-bound3, bound3)
      now_ASQ = given_action_Sequence + np.array([delta_1, delta_2, delta_3])
      return now_ASQ

    def TwoLink(self, dynamics, state, goal, given_action_Sequence):

      # self.control_horizon = 20
      # self.planning_horizon = 50
      # self.OPTM_iter = 210
      # self.control_horizon =  #so much faster 0.124 min
      # self.control_horizon = 20
      self.control_horizon = 8
      self.planning_horizon = 44  #planning_horizon nononononono
      self.OPTM_iter = 65
      self.velocity_weight = 1
      bound1 ,bound2, bound3=0.153, 0.12 , 0.163 # worked
      # bound1 ,bound2, bound3=0.15, 0.10 , 0.163 # worked


      # if goal[1,0] > 0:
      #   # self.control_horizon = 10
      #   # self.planning_horizon = 60
      #   self.control_horizon = 20
      #   self.planning_horizon = 70
      #   # bound1 ,bound2, bound3=0.16, 0.12 , 0.163
      #   bound1 ,bound2, bound3=0.17, 0.14 , 0.163


      delta_1,delta_2,delta_3 =  random.uniform(-bound1, bound1),random.uniform(-bound2, bound2),random.uniform(-bound3, bound3)
      now_ASQ = given_action_Sequence + np.array([delta_1, delta_2, delta_3])
      return now_ASQ


    def ThreeLink(self,dynamics, state, goal, given_action_Sequence):
      self.control_horizon = 10
      self.planning_horizon = 60
      self.OPTM_iter = 100
      vel_magnitude = np.linalg.norm(dynamics.compute_vel_ee(state))
      e_pos = np.linalg.norm(dynamics.compute_fk(state) - goal)
      vel_ee = dynamics.compute_vel_ee(state)
      bound1 ,bound2, bound3= 0.18, 0.12 , 0.11
      # if goal[1,0] > 1:
      #   bound1 ,bound2, bound3= 0.18, 0.12 , 0.11
      delta_1,delta_2,delta_3 =  random.uniform(-bound1, bound1),random.uniform(-bound2, bound2),random.uniform(-bound3, bound3)
      now_ASQ = given_action_Sequence + np.array([delta_1, delta_2, delta_3])
      return now_ASQ

# Manually testing the controller
This part is for you to manually check the performance of your controller before you are ready for it be evaluated by our scoring function.
To test your implementation run the following code. Feel free to play around with the cell or change the num_links / goal positions . You can define your controller however you would like to and then switch on gui to see how close your end effectors get to the goal position

Every time step within the environment is 0.01s, which is defined in the dynamics as `dt`.

The MPC class has a `control_horizon` variable which represents the frequency at which `controller.compute_action()` will be called

In the scoring function you will be evaluated on the distance of your end effector to the goal position and the velocity of the end effector.

In [ ]:
%%script false --no-raise-error
# @title Skip Manually Testing
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time
import math
np.set_printoptions(suppress=True)

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=3,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False

if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()

# Resetting the arm will set its state so that it is in the vertical position,
# and set the action to be zeros
arm.reset()

# Choose the goal position you would like to see the performance of your controller
goal = np.zeros((2, 1))
goal[0, 0] = 2.5
goal[1, 0] = -0.7
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)

# Control loop
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()

    # Measuring distance and velocity of end effector
    pos_ee = dynamics_teacher.compute_fk(state)
    dist = np.linalg.norm(goal-pos_ee)
    vel_ee = np.linalg.norm(arm.dynamics.compute_vel_ee(state))
    print(f'At timestep {s}: Distance to goal: {dist}, Velocity of end effector: {vel_ee}')

    action = controller.compute_action(arm.dynamics, state, goal, action)
    arm.set_action(action)

## Grading and Evaluation for Part 1
Your controller will be graded on 6 tests. 2 tests each for the 1-link ,2-link, and 3-link arms. The arm will start off in the initial state with the arms pointing stright down. The testing criteria depend on the distance and the velocity of the end effectors . Each test will run the robot arm for **5.0 seconds**. At the end of the 5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.1 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`


In [ ]:
# Scoring using score_mpc 14min
controller = MPC()
gui = False

In [ ]:
# DO NOT CHANGE
score_mpc_true_dynamics(controller, gui)


Part1: EVALUATING CONTROLLER (with perfect dynamics)
-----------------------------------------------------
NUM_LINKS: 1
Test  1
Success! :)
 Goal: [ 0.38941834 -0.92106099], Final position: [ 0.39308249 -0.91950321], Final velocity: [0.]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-0.68163876 -0.73168887], Final position: [-0.67940245 -0.73376584], Final velocity: [0.]
score: 1.0/1.0
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [ 0.6814821  -1.61185674], Final position: [ 0.6841665  -1.61213747], Final velocity: [0.02018343]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-1.19286783 -1.28045552], Final position: [-1.18601307 -1.2831327 ], Final velocity: [0.01678971]
score: 1.0/1.0
NUM_LINKS: 3
Test  1
Success! :)
 Goal: [ 1.29444895 -2.36947292], Final position: [ 1.30885101 -2.37127617], Final velocity: [0.32704471]
score: 1.5/1.5
Test  2
Success! :)
 Goal: [-2.10367746 -1.35075576], Final position: [-2.10337541 -1.39810876], Final velocity: [0.07041317]
score: 1.0/1.0
       
---------------

### Part 2.1: Model Architecture
Beyond this point you will be focusing on the 2 link arm only.

We have a base class Model and a subclass for the 2-link arm. The class Model is a base class for our models.  In compute_next_state() method,
you have to use the trick to use joint accelerations to compute the next state similar to what you did in Project 3.

In the `Model2Link` class you will use a neural network to compute the joint accelerations by implementing `compute_qddot()` method. This will take 6 values (2 joint angles, 2 joint velocities and 2 actions applied to the arm) and output 2 joint acceleration values

Do not change the arguments for the `__init__()` method even if you do not use them.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
  def __init__(self, num_links=2, time_step=0.01):
    super(Model, self).__init__()
    self.num_links = num_links
    self.time_step = time_step
    self.fc1 = nn.Linear(6, 450)
    self.fc2 = nn.Linear(450, 247)
    self.fc3 = nn.Linear(247, 88)
    self.fc4 = nn.Linear(88, 2)

  def forward(self, x):
    # print("forward")
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

  def compute_next_state(self, state, qddot):

    x = state[:, :2]

    x_dot = state[:, 2:4]

    x_dot_next = x_dot + qddot * self.time_step

    x_next = x + x_dot * self.time_step + 0.5 * qddot * self.time_step **2


    next_state = torch.concatenate((x_next.T, x_dot_next.T)).T
    return next_state

  def compute_qddot(self, x):
    pass

class Model2Link(Model):
  def __init__(self, time_step):
    torch.manual_seed(42)
    super().__init__(2, time_step)

  def compute_qddot(self, x):
      self.eval()  # Set the model to evaluation mode
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = F.relu(self.fc3(x))
      x = self.fc4(x)
      return x


### Part 2.2: Collect Data
Similar to project 3, we will collect data which will be used to learn a forward model for our 2 link robot arm. Once we have learnt a forward model you will be evaluated on your MPC Controller that uses the learnt dynamics model instead of the true dynamics.

You can modify the collect_data function or write any of your own functions however you choose to. You will be evaluated on the **2 Link Robot**

In [ ]:
# @title data collect function
# Teacher arm with 2 links
from IPython.display import clear_output
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

def generate_points_in_semicircle(r, num_points_x, num_points_y):
    # Create a grid of points in the rectangle that bounds the lower half-circle
    x_values = np.linspace(-r, r, num_points_x)
    y_values = np.linspace(-r, 0, num_points_y)  # Only the lower half (y <= 0)
    grid_x, grid_y = np.meshgrid(x_values, y_values)
    points = np.vstack([grid_x.ravel(), grid_y.ravel()]).T
    points = points[np.where(points[:, 0] ** 2 + points[:, 1] ** 2 <= r ** 2)]

    return points

# point_cloud = np.reshape(generate_points_in_semicircle(2, 13,13), (-1, 2))
# print(point_cloud.shape[0])
# print(point_cloud)
def collect_data(arm):
  random.seed(8848)
  np.random.seed(88481)
  # ---
  # You code goes here. Replace the X, and Y by your collected data
  # Control the arm to collect a dataset for training the forward dynamics.
  # num_samples = 1


  # ---
  per_point = 250 #ms velocity_current

  hor_num_point ,verti_num_point = 13,13
  point_cloud = np.reshape(generate_points_in_semicircle(1.95, hor_num_point,verti_num_point), (-1, 2))

  num_point = point_cloud.shape[0]
  total_sample = per_point*num_point
  num_retake = 0
  X = np.zeros((total_sample, arm.dynamics.get_state_dim() + arm.dynamics.get_action_dim()))
  Y = np.zeros((total_sample, arm.dynamics.get_state_dim()))




  est = 0
  loc = 0

  for i in range(num_point):
    # print(i)
    start = time.time()
    arm = Robot(dynamics_teacher)
    arm.reset()

    state,action,goal = arm.get_state(), np.zeros((2,1)), np.zeros((2,1))
    r = np.random.uniform(low=0.05, high=1.95)
    theta = np.random.uniform(low=np.pi, high=2.0*np.pi)




    goal[0,0] ,goal[1,0] = point_cloud[i][0],point_cloud[i][1]
    arm.goal = goal

    for dt in range(per_point):
      #
      #
      X[loc][:4] ,X[loc][4:] = state.flatten()[:4] ,action.flatten()[:2]
      arm.advance()
      state = arm.get_state()

      if dt % 20 == 0:  #control_horizon 16min
        action = controller.compute_action(arm.dynamics, state, goal, action)


      arm.set_action(action)
      Y[loc] = state.flatten()[:4]

      loc += 1
    end = time.time()
    est = (end - start)
    clear_output(wait=True)  # Clear the output before printing the new value
    print(f"Point {i+1}/{num_point} {(i+1)*per_point} / {total_sample}  {((i+1)*per_point/total_sample*100):.3f} %  Remains: {int(((num_point-i)*est // 3600))} hrs {int( ((num_point-i)*est) % 60)} min")

  return X, Y




In [ ]:
import pickle

# Call the function you have defined above to collect data
X, Y = collect_data(arm)
save_dir = 'dataset'
if not os.path.exists(save_dir):
  os.makedirs(save_dir)

# Save the collected data in the data.pkl file
data = {'X': X, 'Y': Y}
pickle.dump(data, open(os.path.join(save_dir, 'data.pkl'), "wb" ))
# 50min

Point 117/117 29250 / 29250  100.000 %  Remains: 0 hrs 26 min


### Part 2.3: Training the forward model
By now you would be familiar with the basic skeleton of training a forward model.

The starter code already creates the dataset class and loads the dataset with a random 0.8/0.2 train/test split for you. This script should save the model that it trains. You should use a specific procedure for saving, outlined below.

In machine learning, it is a very good practice to save not only the final model but also the checkpoints. Our starter code already configures save_dir for you and for each epoch of your training, you should use the following code to save the model at that epoch.

```
model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
if not os.path.exists(os.path.join(args.save_dir, model_folder_name)):
    os.makedirs(os.path.join(args.save_dir, model_folder_name))
torch.save(model.state_dict(), os.path.join(args.save_dir, model_folder_name, 'dynamics.pth'))
print(f'model saved to {os.path.join(args.save_dir, model_folder_name, "dynamics.pth")}\n')
```
The output from running this code should be a folder as below:

```
models/
    2021-03-24_23-57-50/
        epoch_0001_loss_0.00032930/
            dynamics.pth
        epoch_0002_loss_0.00009413/
            dynamics.pth   
        ...  
```
You can implement the functions below as you please to collect data

In [ ]:

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import os
import numpy as np
# import comet_ml
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import pickle
import torch.optim as optim
import argparse
import time
from torch.utils.tensorboard import SummaryWriter
from IPython.display import clear_output

class DynamicDataset(Dataset):
  def __init__(self, datafile):
    data = pickle.load(open(datafile, 'rb'))
    # print("DynamicDataset")
    # X: (N, 6), Y: (N, 4)
    self.X = data['X'].astype(np.float32)
    self.Y = data['Y'].astype(np.float32)
    delta_t = 0.01
    initial_velocities = self.X[:, 2:4]
    final_velocities = self.Y[:, 2:4]
    delta_v = final_velocities - initial_velocities
    acceleration = delta_v / delta_t
    self.Y = acceleration.astype(np.float32)

  def __len__(self):
    return self.X.shape[0]
  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

def train_forward_model():
  # The ratio of the dataset used for testing
  split = 0.2
  torch.manual_seed(42)


  device = torch.device("cpu")
  dataset = DynamicDataset('/content/dataset/data.pkl')
  dataset_size = len(dataset)
  test_size = int(np.floor(split * dataset_size))
  train_size = dataset_size - test_size
  train_set, test_set = random_split(dataset, [train_size, test_size])

  train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
  test_loader = DataLoader(test_set, batch_size=64, shuffle=True)
  # The name of the directory to save all the checkpoints
  timestr = time.strftime("%Y-%m-%d_%H-%M-%S")
  model_dir = os.path.join('models', timestr)
  np.random.seed(88488)


  model = Model2Link(0.01).to(device)
  # optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.000001)#,weight_decay=0.000001
  optimizer =torch.optim.AdamW(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001, amsgrad=False)
  scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)
  criterion = nn.MSELoss()

  epochs = 70 #50
  best_test_loss = 100
  model_path = None


  for epoch in range(1, epochs + 1):
      # Train the model
      model.train()
      train_loss = 0
      for X_batch, Y_batch in train_loader:
          X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
          optimizer.zero_grad()  # Clear the gradients
          output = model(X_batch)  # Forward pass
          loss = criterion(output, Y_batch)  # Compute the loss
          loss.backward()  # Backward pass
          optimizer.step()  # Update the weights
          # scheduler.step()
          train_loss += loss.item() * X_batch.size(0)
      train_loss /= len(train_loader.dataset)


      # Evaluate the model
      model.eval()
      test_loss = 0
      with torch.no_grad():
          i=0
          for X_batch, Y_batch in test_loader:
              X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
              output = model(X_batch)
              loss = criterion(output, Y_batch)
              test_loss += loss.item() * X_batch.size(0)
      test_loss /= len(test_loader.dataset)

      print(f'Epoch: {epoch}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f}')
      if test_loss < best_test_loss:
          best_test_loss = test_loss
          # print(test_loss,best_test_loss)
          model_folder_name = f'epoch_{epoch:04d}_loss_{test_loss:.8f}'
          if not os.path.exists(os.path.join(model_dir, model_folder_name)):
              os.makedirs(os.path.join(model_dir, model_folder_name))
          torch.save(model.state_dict(), os.path.join(model_dir, model_folder_name, 'dynamics.pth'))
          model_path = os.path.join(model_dir, model_folder_name, 'dynamics.pth')
  print('best: ',model_path)
  return model_path



In [ ]:
model_path = train_forward_model()

Model(nn.Module)
Epoch: 1, Train Loss: 6.13736338, Test Loss: 5.01034185
Epoch: 2, Train Loss: 4.95110078, Test Loss: 3.88426359
Epoch: 3, Train Loss: 3.72666230, Test Loss: 2.92026531
Epoch: 4, Train Loss: 2.72692119, Test Loss: 2.06196360
Epoch: 5, Train Loss: 1.84187958, Test Loss: 1.32430775
Epoch: 6, Train Loss: 1.16951338, Test Loss: 0.95151146
Epoch: 7, Train Loss: 0.75935231, Test Loss: 0.56498036
Epoch: 8, Train Loss: 0.53089545, Test Loss: 0.41556381
Epoch: 9, Train Loss: 0.39375916, Test Loss: 0.34810124
Epoch: 10, Train Loss: 0.31689921, Test Loss: 0.28012965
Epoch: 11, Train Loss: 0.25560346, Test Loss: 0.21089424
Epoch: 12, Train Loss: 0.21446703, Test Loss: 0.19011343
Epoch: 13, Train Loss: 0.17973654, Test Loss: 0.16154590
Epoch: 14, Train Loss: 0.16057965, Test Loss: 0.14529529
Epoch: 15, Train Loss: 0.14299180, Test Loss: 0.12773605
Epoch: 16, Train Loss: 0.12936295, Test Loss: 0.11245243
Epoch: 17, Train Loss: 0.11723921, Test Loss: 0.10523451
Epoch: 18, Train Loss: 

In [ ]:
# @title SKip
%%script false --no-raise-error
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

gui=False
print(model_path)
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

### Part 2.4: Completing ArmDynamicsStudent

After you are done with training, you need to complete ArmDynamicsStudent class following the comments below to load the saved checkpoint (in function init_model) and then use it to predict the new state given the current state and action (in function dynamics_step). Please do not modify the arguments to those functions, even though you might not use all of them.

In [ ]:
from arm_dynamics_base import ArmDynamicsBase

class ArmDynamicsStudent(ArmDynamicsBase):
    def init_model(self, model_path, num_links, time_step, device):
        # ---
        # Your code hoes here
        # Initialize the model loading the saved model from provided model_path
        self.model = Model2Link(time_step)  # Initialize your neural network model
        self.model.load_state_dict(torch.load(model_path, map_location=device))  # Load the trained model weights
        self.model.to(device)
        self.model.eval()
        self.device = device
        self.model_loaded = True
        np.random.seed(42)

    def dynamics_step(self, state, action, dt):
        if self.model_loaded:
            # Your code goes here
            action,state = action.flatten()[:2] , state.flatten()



            state_action = np.concatenate((state, action), axis=0).flatten()
            # print(state_action.dtype)
            state_action_tensor = torch.from_numpy(state_action).float().to(self.device)
            with torch.no_grad():  # No need to compute gradients for inference
                new_state = self.model(state_action_tensor)
            acceleration = new_state.cpu().numpy().flatten()  # Convert to numpy array and reshape if necessary


            new_velocities = state[2:4] + acceleration * dt  # v = v_o + a * dt
            new_positions = state[0:2] + state[2:4] * dt + 0.5 * acceleration * dt**2  # x = x_o + v_o * dt + 0.5 * a * dt^2


            new_state = np.concatenate((new_positions, new_velocities)).reshape(-1, 1)


            return new_state

        else:
            return state


In [ ]:
%%script false --no-raise-error
!rm -r /content/models/
# !rm -r /content/runs

### Manually Testing the MPC Controller with the learnt dynamics model
We will now use the learnt dynamics model that you have trained. The model is loaded in the dynamics.init_model method. You can modify the goal positions to see how well is the controller performing similar to what you did before. Feel free to play around with the code in this cell to test your performance before the grading part.


In [ ]:
# @title SKIPPED Manually Testing the MPC Controller with the learnt dynamics model
%%script false --no-raise-error
import sys
import numpy as np
from arm_dynamics_teacher import ArmDynamicsTeacher
from robot import Robot
from render import Renderer
from score import *
import torch
import time

# Teacher arm with 3 links
dynamics_teacher = ArmDynamicsTeacher(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)

arm = Robot(dynamics_teacher)
arm.reset()

gui = False
action = np.zeros((arm.dynamics.get_action_dim(), 1))
if gui:
  renderer = Renderer()
  time.sleep(1)

# Controller
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
device = torch.device('cpu')

# model_path should have the path to the best model that you have trained so far
# which you would like to use for testing the controller
model_path = None
dynamics_student.init_model(model_path, 2, 0.01, device)

# Control loop
action = np.zeros((arm.dynamics.get_action_dim(), 1))
goal = np.zeros((2, 1))
goal[0, 0] = 2.7
goal[1, 0] = 0.5
arm.goal = goal

dt = 0.01
time_limit = 2.5
num_steps = round(time_limit/dt)
for s in range(num_steps):
  t = time.time()
  arm.advance()

  if gui:
    renderer.plot([(arm, "tab:blue")])
  time.sleep(max(0, dt - (time.time() - t)))

  if s % controller.control_horizon==0:
    state = arm.get_state()
    action = controller.compute_action(dynamics_student, state, goal, action)
    arm.set_action(action)


## Grading and Evaluation of Part 2
You will be evaluated on how well your controller+learnt dynamics works together. The scoring functions consists of 16 random test goals all of which will be below the x axis and between 0.05 to 1.95 lengths away from the origin.
The controller will call the compute_action method from your MPC class and apply the action for 10 timesteps
```
action = controller.compute_action(dynamics_student, state, goal, action)
```

Each test will run the robot arm for **2.5 seconds**. At the end of the 2.5 seconds the test will be:

A success if your end effectors meet this criteria:
`distance_to_goal < 0.2 and vel_ee < 0.5`

A partial success if your end effectors meet this criteria:
`distance_to_goal < 0.3 and vel_ee < 0.5`

You need 15 out of the 16 tests to succeed to get a full score

In [ ]:
controller = MPC()
dynamics_student = ArmDynamicsStudent(
    num_links=2,
    link_mass=0.1,
    link_length=1,
    joint_viscous_friction=0.1,
    dt=0.01)
model_path = "dynamics.pth"
gui=False

In [ ]:
# DO NOT CHANGE
score_mpc_learnt_dynamics(controller, dynamics_student, model_path, gui)

Part2: EVALUATING CONTROLLER + LEARNED DYNAMICS
-----------------------------------------------
NUM_LINKS: 2
Test  1
Success! :)
 Goal: [-1.14326324 -0.20951344], Final position: [-1.1201205 -0.1751576], Final velocity: [0.06410138]
score: 0.5/0.5
Test  2
Success! :)
 Goal: [-0.40200574 -0.01780895], Final position: [-0.29792699  0.03093561], Final velocity: [0.00472582]
score: 0.5/0.5
Test  3
Success! :)
 Goal: [-1.59201924 -0.6398353 ], Final position: [-1.56908248 -0.67039887], Final velocity: [0.0463526]
score: 0.5/0.5
Test  4
Success! :)
 Goal: [ 0.38623125 -0.14304274], Final position: [ 0.31689319 -0.07313461], Final velocity: [0.02250229]
score: 0.5/0.5
Test  5
Success! :)
 Goal: [ 0.24047506 -1.25765853], Final position: [ 0.23629722 -1.25868794], Final velocity: [0.00981923]
score: 0.5/0.5
Test  6
Success! :)
 Goal: [-0.48097545 -0.56348721], Final position: [-0.4653496  -0.54621938], Final velocity: [0.01743063]
score: 0.5/0.5
Test  7
Success! :)
 Goal: [-0.03710098 -1.66880

Time Limit
*   MPC evaluation (Part 1): 30 minutes
*   Data Collection with MPC (Part 2): 1 hour 20 minutes
*   Training the forward dynamics model (Part 3): 40 minutes

Note: Time limits are general guidelines. While your code must run within the allotted time, you should be able to achieve full score with a much shorter runtime


Hints and Suggestions:
1. You can use your MPC Controller in your data collection to gather better training samples
2. A good cost function to evaluate your trajectory in MPC is very important and you can use both distance and velocity metrics to define the cost function.
3. As mentioned in the lecture, a constant torque with pseudo gradients seems to work well for this project. You can also use multiple delta values to gather more trajectories to choose from.
4. Since we are passing the MPC object to the controller you can instantiate the MPC class with different parameters like the planning horizon, delta values etc.
5. To speed up data collection, avoid using np.concatenate(), np.stack() or np.append() like functions on your X and Y arrays. Instead, initialize X and Y arrays with all zeros using the correct shape and then fill in the values one by one. This is much faster in numpy. Using Ray is optional